# run_chatbot

In [ ]:
# import

import pickle
import tflearn
import numpy as np
import tensorflow as tf
import random
import json
import nltk
from nltk.stem.lancaster import LancasterStemmer

# Load NLTK data
# nltk.download('punkt')

# Initialize LancasterStemmer
stemmer = LancasterStemmer()

# Load training data

In [ ]:
print("Loading Pickle.....")
data = pickle.load(open("training_data", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# Reset the graph data
# tf.reset_default_graph()

with open('intents.json') as json_data:
    intents = json.load(json_data)

# Load the model
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

print("Loading the Model......")
model.load('model.tflearn')

# Define functions for sentence cleaning and bag of words creation

In [ ]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words


# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)

# Set the error threshold
ERROR_THRESHOLD = 0.25


# Define functions for classification and response
def classify(sentence):
    
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    
    # Exclude those results which are Below Threshold
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence):
    results = classify(sentence)
    
    # if Classification is Done then Find the Matching Tag.
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return random.choice(i['responses'])
            results.pop(0)


# Interaction loop

In [ ]:
while True:
    input_data = input()
    print("You- " + input_data)

    # Check for a stop command
    if input_data.lower() == 'exit':
        print("--- Chatbot Stopped ---")
        break

    answer = response(input_data)

    # Check if answer is not None before printing
    if answer is not None:
        print("Bot- " + answer)
    else:
        print("Bot- I'm sorry, I didn't understand that.")